# Chat History Reduction with Agent Scratchpad in Semantic Kernel

This notebook demonstrates how to use Semantic Kernel's Chat History Reduction feature along with an agent scratchpad for maintaining context across conversations. This is essential for building efficient AI agents that can handle long conversations without exceeding token limits.

## What You'll Learn:
1. **Chat History Reduction**: How to automatically summarize conversation history to manage token usage
2. **Agent Scratchpad**: A persistent memory system for tracking user preferences and completed tasks
3. **Token Usage Tracking**: Monitor how token usage changes with and without history reduction

## Prerequisites:
- Azure OpenAI setup with environment variables configured
- Understanding of basic agent concepts from previous lessons

## Import Required Packages

In [1]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## Understanding Agent Scratchpad

### What is an Agent Scratchpad?

An **Agent Scratchpad** is a persistent memory system that agents use to:
- **Track completed tasks**: Record what has been done for the user
- **Store user preferences**: Remember likes, dislikes, and requirements
- **Maintain context**: Keep important information accessible across conversations
- **Reduce redundancy**: Avoid asking the same questions repeatedly

### How it Works:
1. **Write Operations**: Agent updates the scratchpad after learning new information
2. **Read Operations**: Agent consults the scratchpad when making decisions
3. **Persistence**: Information survives even when chat history is reduced

Think of it as the agent's personal notebook that complements the conversation history.

## Environment Configuration

In [2]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

✅ Azure OpenAI service configured


## Create the Agent Scratchpad Plugin

This plugin allows the agent to read and write to a persistent scratchpad file.

In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

📝 Scratchpad plugin created


## Initialize Chat History Reducer

The ChatHistorySummarizationReducer automatically summarizes conversation history when it exceeds a threshold.

In [26]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 20    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

🔄 Chat History Reducer configured:
   - Reduction triggered at: 20 messages
   - Reduces history to: 5 messages


## Create the Vacation Planning Agent

This agent will help users plan vacations while maintaining context through the scratchpad.

In [27]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    IMPORTANT SCRATCHPAD USAGE:
    1. ALWAYS read the scratchpad at the start of planning to check for existing user preferences
    2. When you learn new preferences (destination types, activities, budget, travel style), 
       update the scratchpad with category 'preferences'
    3. When you complete a task (like creating an itinerary), update the scratchpad with category 'tasks'
    4. Before creating a new itinerary, ALWAYS check the scratchpad for user preferences
    
    PLANNING PROCESS:
    1. Ask about destination preferences if not in scratchpad
    2. Ask about travel dates, budget, and activities they enjoy
    3. Create detailed day-by-day itineraries
    4. Be ready to adapt plans based on user feedback
    
    Format itineraries with clear structure:
    - Day headers
    - Morning/Afternoon/Evening activities
    - Estimated costs
    - Travel tips
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with scratchpad access")

🤖 Vacation Planning Agent created with scratchpad access


## Helper Functions for Display and Token Tracking

In [28]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

📊 Token tracking initialized


## Run the Vacation Planning Conversation

Now let's run through a complete conversation demonstrating:
1. Initial planning request
2. Preference gathering
3. Itinerary creation
4. Location change
5. Chat history reduction
6. Scratchpad usage

In [29]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status
        print(f"📝 Thread has {len(thread)} messages")

        # Show message types to understand what's in the thread
        if i == 0:  # Show details for first message to understand structure
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types in thread: {message_types}")

        # Check if reduction happened
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

        # Attempt reduction manually to see if it works
        is_reduced = await thread.reduce()
        if is_reduced:
            print(
                f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
            token_tracker.mark_reduction(message_count + 1)

            # Show summary if available
            async for msg in thread.get_messages():
                if msg.metadata and msg.metadata.get("__summary__"):
                    display_message("System Summary", str(
                        msg.content), "#ff6b6b")  # Red for system summary
                    break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    return thread

# Run the conversation
thread = await run_vacation_planning()

🚀 Starting Vacation Planning Session



📝 Thread has 2 messages
   Message types in thread: ['user', 'assistant']


📝 Thread has 6 messages


📝 Thread has 10 messages


📝 Thread has 14 messages


📝 Thread has 16 messages


📝 Thread has 18 messages


📝 Thread has 20 messages


📝 Thread has 22 messages
   ⚠️ Thread length (22) exceeds threshold (20)

--- Token Usage Analysis ---



--- Final Scratchpad Contents ---


## Analyzing the Results

Let's analyze what happened during our conversation:

In [19]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

📊 Total Token Usage Summary

Total Prompt Tokens: 42,084
Total Completion Tokens: 8,333
Total Tokens Used: 50,417

💡 Note: The reduction impact is shown in the chart above.
Look for the dramatic drop in prompt tokens after the REDUCTION marker.
This shows how chat history summarization reduces the context size for future messages.


## Key Takeaways

### 1. Chat History Reduction
- **Automatic Triggering**: Reduction happens when message count exceeds threshold
- **Token Savings**: Significant reduction in token usage after summarization
- **Context Preservation**: Important information is retained in summaries

### 2. Agent Scratchpad Benefits
- **Persistent Memory**: User preferences survive history reduction
- **Task Tracking**: Agent maintains record of completed work
- **Improved Experience**: No need to repeat preferences

### 3. Token Usage Patterns
- **Linear Growth**: Tokens increase with each message
- **Dramatic Drop**: Reduction significantly decreases token count
- **Sustainable Conversations**: Enables longer interactions within limits

## Cleanup

Clean up the scratchpad file created during this demo:

In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# Summary

Congratulations! You've successfully implemented an AI agent with advanced context management capabilities:

## What You've Learned:
- **Chat History Reduction**: Automatically summarize conversations to manage token limits
- **Agent Scratchpad**: Implement persistent memory for user preferences and completed tasks
- **Token Management**: Track and optimize token usage in long conversations
- **Context Preservation**: Maintain important information across conversation reductions

## Real-World Applications:
- **Customer Service Bots**: Remember customer preferences across sessions
- **Personal Assistants**: Track ongoing projects and user habits
- **Educational Tutors**: Maintain student progress and learning preferences
- **Healthcare Assistants**: Keep patient history while respecting token limits

## Next Steps:
- Implement more sophisticated scratchpad schemas
- Add database storage for multi-user scenarios
- Create custom reduction strategies for domain-specific needs
- Combine with vector databases for semantic memory search
- Build agents that can resume conversations days later with full context